In [1]:
import pandas as pd

df = pd.read_csv("training_data.csv")
df.head()

,reference,translation
0,shite i dont think theyre very happy,i dont think theyre very happy
1,if you want to understand animals i mean i mea...,if you want to understand game i mean really u...
2,sonia that smells awful,sonio that is a terrible thing
3,sam dolans father is out of his mind,sam dolans father checks out
4,theres a guy out there whos fucking serious,this is one very serious boy we have out there


In [2]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_pairs(toxic, non_toxic):
    inputs = tokenizer(toxic, padding=True, truncation=True, return_tensors="pt")
    targets = tokenizer(non_toxic, padding=True, truncation=True, return_tensors="pt")
    return inputs, targets

# Example of tokenizing a single pair
tokenized_example = tokenize_pairs(df['reference'][0], df['translation'][0])

In [3]:
tokenized_example

({'input_ids': tensor([[ 101, 4485, 2063, 1045, 2123, 2102, 2228, 2027, 2890, 2200, 3407,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[ 101, 1045, 2123, 2102, 2228, 2027, 2890, 2200, 3407,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])})

In [4]:
from transformers import DistilBertForSequenceClassification


# Initialize BERT2BERT from pre-trained checkpoints
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased').to("cuda")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

num_epochs = 10

def collate_fn(batch):
    input_ids = [item[0]['input_ids'].squeeze(0) for item in batch]  # Remove the batch dimension
    attention_mask = [item[0]['attention_mask'].squeeze(0) for item in batch]  # Remove the batch dimension
    labels = [item[1]['input_ids'].squeeze(0) for item in batch]  # Remove the batch dimension

    # Pad the sequences
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask_padded = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=tokenizer.pad_token_id)

    # Ensure that labels are padded to the max length of input_ids if they are shorter
    max_len = input_ids_padded.shape[1]
    labels_padded = pad_sequence(labels_padded, batch_first=True, padding_value=tokenizer.pad_token_id)

    return {
        'input_ids': input_ids_padded, 
        'attention_mask': attention_mask_padded,
        'labels': labels_padded
    }

# Create a PyTorch dataset
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, data):
        self.tokenizer = tokenizer
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        toxic, non_toxic = self.data.iloc[idx]
        inputs = tokenizer(toxic, return_tensors="pt")
        targets = tokenizer(non_toxic, return_tensors="pt")
        return inputs, targets

dataset = TextDataset(tokenizer, df)
loader = DataLoader(dataset, batch_size=1728, shuffle=True, collate_fn=collate_fn)
device = "cuda"
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Define a training function
def train(model, dataloader):
    model.train()
    optimizer = torch.optim.Adam(model.parameters())

    for epoch in range(num_epochs):
        for i, batch in enumerate(dataloader):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            optimizer.zero_grad()
            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            if i % 10 == 0:
                print(i, len(dataloader), loss)
        print(epoch, loss)

# Call the train function
train(model, loader)

OutOfMemoryError: CUDA out of memory. Tried to allocate 532.00 MiB. GPU 0 has a total capacty of 79.32 GiB of which 136.81 MiB is free. Including non-PyTorch memory, this process has 79.18 GiB memory in use. Of the allocated memory 77.65 GiB is allocated by PyTorch, and 319.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from transformers import BertTokenizerFast

def evaluate(model, dataloader):
    model.eval()
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    predictions = []
    references = []

    for batch in dataloader:
        inputs, targets = batch
        with torch.no_grad():
            outputs = model.generate(**inputs)
        
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_targets = tokenizer.batch_decode(targets['input_ids'], skip_special_tokens=True)
        predictions.extend(decoded_preds)
        references.extend(decoded_targets)

evaluate(model, validation_loader)  # Assuming you have a validation_loader set up similarly to `loader`